# RAG skeleton 
In the following we'll have the skeleton of the RAG system. It is going to be a very basic implementation, that we are going to expand on later milestones.

In [ ]:
import os
import json
from pathlib import Path
from llama_index.llms.ollama import Ollama
from llama_index.core import VectorStoreIndex
from llama_index.core.embeddings import resolve_embed_model
from llama_index.readers.json import JSONReader
from llama_index.core.node_parser import JSONNodeParser
from llama_index.readers.file import FlatReader

### VectorDB

In [ ]:
import psycopg2

with open('./db.txt', "r") as file:
    connection_string = file.read()

conn = psycopg2.connect(connection_string)

conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [ ]:
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import textwrap


url = make_url(connection_string)
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="rag",
    embed_dim=1024,  # openai embedding dimension
)


#### Loading and Indexing
Load the data in order to make the documents' embeddings

In [ ]:
embed_model = resolve_embed_model("local:BAAI/bge-m3")

In [ ]:
# set a path to folder containing all the json files
DATA_PATH = "./data/"

# setting up reader, parser, and llm
reader = JSONReader()

# parser = JSONNodeParser()     # if we want to split the documents into nodes
llm = Ollama(model="mistral", request_timeout=180.0) 

In [ ]:

# creating the documents out of the json files
documents = []
for filename in os.listdir(DATA_PATH):
    if filename.endswith(".json"):
        file_path = os.path.join(DATA_PATH, filename)
        documents.extend(FlatReader().load_data(Path(file_path)))     # if we want to load the data to then split it into nodes
        # documents.extend(reader.load_data(input_file=file_path))
parser = JSONNodeParser(include_metadata=True,
                        include_prev_next_rel=True)

# nodes = parser.get_nodes_from_documents(documents)            # if we want to split documents into nodes


In [ ]:
len(documents)

### Document splitting

if you want to use a simple node parser

if you want to have control on the entire pipeline (can also choose the chunk size)

In [ ]:
import nest_asyncio
import nltk
nltk.download('punkt_tab')
nest_asyncio.apply()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor

from llama_index.core.node_parser import TokenTextSplitter

from llama_index.core.ingestion import IngestionPipeline


text_splitter = TokenTextSplitter(
    # separator=" ", 
    chunk_size=512, 
    chunk_overlap=128
)

#if you wanna create some custom extractor

# class CustomExtractor(BaseExtractor):
#     def extract(self, nodes):
#         metadata_list = [
#             {
#                 "custom": (
#                     node.metadata["document_title"]
#                     + "\n"
#                     + node.metadata["excerpt_keywords"]
#                 )
#             }
#             for node in nodes
#         ]
#         return metadata_list

transformations = [
    text_splitter,
    # TitleExtractor(nodes=3,llm=llm),
    # QuestionsAnsweredExtractor(questions=2,llm=llm),
    # SummaryExtractor(summaries=["prev", "self"],llm=llm),
    # KeywordExtractor(keywords=4,llm=llm),
    EntityExtractor(prediction_threshold=0.5,llm=llm),
]


pipeline = IngestionPipeline(
    transformations=transformations
)

nodes = pipeline.run(
    documents=documents,
    in_place=True,
    show_progress=True,
)

In [ ]:
len(nodes)

In [ ]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

### Storing
Load into the vectorDB

In [ ]:
vector_store.add(nodes)

### Querying (milestone 2)

In [ ]:
query_str = "General Summarized Overview Large Capacity Cutting Machine 2?"

query_embedding = embed_model.get_query_embedding(query_str)

In [ ]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [ ]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

In [ ]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [ ]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [ ]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=5
)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [ ]:
response = query_engine.query(query_str)

In [ ]:
response

### Querying strategy

In [ ]:
# if we work with nodes
#vector_index = VectorStoreIndex.from_documents(nodes, embed_model=embed_model)

In [ ]:
# if we work with documents
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, show_progress=True)

we use top-k similarity strategy to get the k most similar documents

In [ ]:
query_engine = vector_index.as_query_engine(llm=llm, verbose=True, similarity_top_k=2)
retriever = vector_index.as_retriever(verbose=True)

### Evaluation
We test the RAG system with some queries regarding the data in the json files

In [ ]:
result = query_engine.query("What was the average  of Assembly Machines?")
print(result)

In [ ]:
result = query_engine.query("What was the average consumption of machines?")
print(result)

In [ ]:
result = query_engine.query("List the conspumption for each machine in March 2024?")
print(result)

In [ ]:
result = query_engine.query("General Summarized Overview Large Capacity Cutting Machine 2?")
print(result)

In [ ]:
result = query_engine.query("Which machine has higher idle time")
print(result)

In [ ]:
retriever.retrieve("General Summarized Overview Assembly Machine 1?")


In [ ]:
result = query_engine.query("Which one was more effective and productive: Medium Capacity machine 1 vs Medium Capacity machine 2?")
print(result)

### JSON Outputs
The idea is to define 2 prompts, one for each type of query: `report generation` and `KPI suggestion`.

#### Prompt Definition

In [ ]:
# 1. Prompt for Generating New KPIs:
kpi_json_prompt = """
You are a specialized assistant that only outputs answers in JSON format. 

Analyze the documents and provide new KPI suggestions. Use the JSON format below:
{
  "KPIs": [
    {
      "name": "<KPI Name>",
      "description": "<Brief description of the KPI>",
      "formula": "<Mathematical Formula to calculate the KPI using existing variables>"
    },
    ...
  ]
}
"""

In [ ]:
# 2. Prompt for Generating Machine Reports:
report_json_prompt = """
You are a specialized assistant that only outputs answers in JSON format.

Based on the monthly aggregated KPI data provided, generate a detailed report for the specified machine and month. Use the JSON format below:
{
    "MachineBehaviorReport": {
        "machine": "<Machine Name>",
        "month": "<Month>",
        "kpi_analysis": [
            {
                "kpi": "<KPI Name>",
                "analysis": "<Analysis of the KPI based on min, max, and average values>"
            },
            ...
        ],
        "overall_summary": "<High-level summary of machine performance>"
    }
}
"""

In [ ]:
# More detailed version of the prompt for generating machine reports
report_json_prompt2 = """
You are a specialized assistant that only outputs answers in JSON format.

Based on the monthly aggregated KPI data provided, generate a detailed report for the specified machine and month. The report must include the following:

1. The name of the machine and the month being analyzed.
2. An analysis of each KPI (listed below), comparing its minimum, maximum, and average values. Identify:
   - Notable patterns or trends (e.g., consistently high or low values).
   - Significant deviations (e.g., high max values with low averages).
   - Missing or zero values, and their implications.
3. An overall summary of the machine's performance for the month, including conclusions about efficiency, potential issues, and general observations.

KPIs to analyze:
- average_cycle_time
- bad_cycles
- consumption
- consumption_idle
- consumption_working
- cost
- cost_idle
- cost_working
- cycles
- good_cycles
- idle_time
- offline_time
- power
- working_time

Respond in the following JSON format:

{
    "MachineBehaviorReport": {
        "machine": "<Machine Name>",
        "month": "<Month>",
        "kpi_analysis": [
            {
                "kpi": "<KPI Name>",
                "analysis": "<Analysis of the KPI based on min, max, and average values>"
            },
            ...
        ],
        "overall_summary": "<High-level summary of machine performance>"
    }
}
"""

#### Query Classification
Given a query we should find a way to choose the proper prompt.

In [ ]:
def classify_query(query):
    kpi_keywords = ["KPI", "KPIs", "metrics", "new", "suggest"]
    report_keywords = ["report", "behavior", "trend", "machine"]

    if any(keyword in query.lower() for keyword in kpi_keywords):
        return "kpi"
    elif any(keyword in query.lower() for keyword in report_keywords):
        return "report"
    else:
        return "unknown"

#### Response Function

In [ ]:
def get_response(query):
    # Classify the query type
    query_type = classify_query(query)
    
    # Select the appropriate prompt
    if query_type == "kpi":
        prompt = kpi_json_prompt
    elif query_type == "report":
        prompt = report_json_prompt
    else:
        prompt = """ """  # no prompt if uncertain / a different type of query was made

    # Pass the query and prompt to the model
    response = query_engine.query(prompt + "\nQuery: " + query)

    # # Validate the response if it's a 'kpi' or 'report' query
    # if query_type in ["kpi", "report"]:
    #     try:
    #         # Try parsing the response as JSON
    #         response_json = json.loads(response)
    #         return response_json
    #     except json.JSONDecodeError:
    #         # Return an error if the response is not valid JSON
    #         return {"error": "Invalid JSON response from model.", "raw_response": response}
    
    # If the query is not of type 'kpi' or 'report', return the raw response
    return response


#### Examples of Usage

In [ ]:
query = "Generate a performance report for the machine 'Large Capacity Cutting Machine 2' for March 2024."
response = get_response(query)

print(response)  

In [ ]:
query = "Suggest new KPIs based on the provided data."
response = get_response(query)

print(response)

In [ ]:
query = "Generate a new KPI to monitor the production efficiency."
response = get_response(query)

print(response)

In [ ]:
query = "What was the behavior of the Laser Cutter in March 2024?"
response = get_response(query)

print(response)

In [ ]:
query = "General Summarized Overview of the Laser Cutter?" # this won't be classified as 'report'
response = get_response(query)

print(response)

In [ ]:
query = "Generate a report on the general behavior of the Riveting Machine and Laser Cutter in March 2024."
response = get_response(query)

print(response)

In [ ]:
query = "How many machines are there?"
response = get_response(query)

print(response)